In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from scipy.stats import spearmanr
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import resample
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [44]:
# load balanced data
data_balanced = pd.read_csv('ToN-IoT_balanced_smote.csv')

In [16]:
# train an lstm model on the data
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn import metrics
import time
X = data_balanced.drop(['label','type'], axis=1)
Y = data_balanced['label']

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# convert Y to int
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# reshape the data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
# Y_train = np.asarray(Y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
# Y_test = np.asarray(Y_test).astype(np.float32)

# define the model and add recurrent dropout
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
start = time.time()
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), shuffle=True)
end = time.time()
print("Time taken to train the model: ", end-start)

Epoch 1/10
75000/75000 [==============================] - 117s 2ms/step - loss: 0.0776 - accuracy: 0.9776 - val_loss: 0.0590 - val_accuracy: 0.9838
Epoch 2/10
75000/75000 [==============================] - 115s 2ms/step - loss: 0.0636 - accuracy: 0.9826 - val_loss: 0.0589 - val_accuracy: 0.9827
Epoch 3/10
75000/75000 [==============================] - 115s 2ms/step - loss: 0.0610 - accuracy: 0.9836 - val_loss: 0.0549 - val_accuracy: 0.9845
Epoch 4/10
75000/75000 [==============================] - 121s 2ms/step - loss: 0.0595 - accuracy: 0.9846 - val_loss: 0.0520 - val_accuracy: 0.9859
Epoch 5/10
75000/75000 [==============================] - 116s 2ms/step - loss: 0.0601 - accuracy: 0.9854 - val_loss: 0.0526 - val_accuracy: 0.9864
Epoch 6/10
75000/75000 [==============================] - 115s 2ms/step - loss: 0.0629 - accuracy: 0.9859 - val_loss: 0.0511 - val_accuracy: 0.9860
Epoch 7/10
75000/75000 [==============================] - 112s 1ms/step - loss: 0.0578 - accuracy: 0.9861 - val_

In [17]:
start_time = time.time()
y_pred = model.predict(X_test)
end_time = time.time()
y_pred = (y_pred > 0.5)
print('Classification Report')
print(metrics.classification_report(Y_test, y_pred))
print('Accuracy:')
print(metrics.accuracy_score(Y_test, y_pred))
print('Time taken to predict: {} seconds'.format(end_time - start_time))

18750/18750 [==============================] - 9s 489us/step
Classification Report
              precision    recall  f1-score   support

           0       0.98      0.88      0.93     59762
           1       0.99      1.00      0.99    540238

    accuracy                           0.99    600000
   macro avg       0.98      0.94      0.96    600000
weighted avg       0.99      0.99      0.99    600000

Accuracy:
0.9863066666666667
Time taken to predict: 12.159243106842041 seconds


In [40]:
# save model
model.save('ton_bin_lstm_smote.h5')